# Comparison of BedMachine3 filled with GEBCO vs IBCSOv2

In [3]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import matplotlib.path as mpath
import cmocean
import pickle

### Functions

In [4]:
def prepare_plot(figsize=(15, 15), dpi=300):
    """This function returns prepared axes for the polar plot.
    
    Usage:
        fig, ax = prepare_plot()
    """
    fig = plt.figure(figsize=figsize)
    ax = plt.axes(projection=ccrs.SouthPolarStereo())
    ax.set_extent([-180, 180, -90, -50], ccrs.PlateCarree())
    ax.set_boundary(map_circle, transform=ax.transAxes)
    # ax.coastlines(); 
    gl = ax.gridlines(draw_labels=True, ylocs=[]);#, color='w', lw=2);
    return fig, ax

### Load files

In [ ]:
coord      = xr.open_dataset('/home/users/birgal/data/bathymetry/coordinates_AIS.nc').squeeze()
Bed_IBCSO  = xr.open_dataset('/home/users/birgal/data/bathymetry/eORCA025_BedMachine3_IBCSO_AIS.nc')
Bed_GEBCO  = xr.open_dataset('/home/users/birgal/data/bathymetry/eORCA025_BedMachine3_GEBCO_AIS.nc')
# bed   = xr.open_dataset('/gws/nopw/j04/terrafirma/kaight/input_data/topo/BedMachineAntarctica-v3.nc')
# BedMachine = xr.open_dataset('/home/users/birgal/eORCA025_BedMachine3_AIS.nc')
# IBCSO      = xr.open_dataset('/home/users/birgal/data/input_data/topo/IBCSO_v2_bed_WGS84.nc')
# Bed_IBCSOt = xr.open_dataset('/home/users/birgal/eORCA025_BedMachine3_IBCSO_AIS_test.nc')

In [ ]:
# Load polygon from disc
with open('/home/users/birgal/data/IBCSO_isobath_1000m_polygon', "rb") as poly_file:
    isobath_1000m_polygon = pickle.load(poly_file)

### Main

In [3]:
# Definitions used for plotting

# To make circular maps:
theta = np.linspace(0, 2*np.pi, 100)
map_circle = mpath.Path(np.vstack([np.sin(theta), np.cos(theta)]).T * 0.5 + [0.5, 0.5])

# Isobath 2000 m polygon
x_iso, y_iso = isobath_1000m_polygon.exterior.xy
x_iso1 = np.ma.masked_where(np.array(x_iso) < 0, np.array(x_iso)) # need to separate longitudes because otherwise matplotlib plotting artifact
y_iso1 = np.ma.masked_where(np.array(x_iso) < 0, np.array(y_iso))
x_iso2 = np.ma.masked_where(np.array(x_iso) >= 0, np.array(x_iso))
y_iso2 = np.ma.masked_where(np.array(x_iso) >= 0, np.array(y_iso))

#### BedMachinev3 filled with IBCSOv2 versus GEBCO

Povl & Emma suggest GEBCO is probably more reliable overall, but BedMachine3 uses IBCSOv1, so it may be more consistent to use IBCSO to fill the gaps from BedMachine3. I plot the difference between the model bathymetries produced with the two different fill datasets below.

In [ ]:
fig, ax = plt.subplots(1,3, figsize=(20,15), subplot_kw={'projection': ccrs.SouthPolarStereo()})#, dpi=1000)

x, y = np.meshgrid(IBCSO['lon'], IBCSO['lat'])
for axis in ax.ravel():
    axis.set_extent([-180, 180, -90, -50], ccrs.PlateCarree())
    axis.set_boundary(map_circle, transform=axis.transAxes)
    gl = axis.gridlines(draw_labels=True, ylocs=[]);
    # add 2000 m isobath contour to each of the plots 
    axis.plot(x_iso1, y_iso1, c='k', zorder=4, linewidth=1.2, transform=ccrs.PlateCarree())
    axis.plot(x_iso2, y_iso2, c='k', zorder=4, linewidth=1.2, transform=ccrs.PlateCarree())

ax[0].set_title('BedMachine bathy filled with IBCSOv2')
ax[1].set_title('BedMachine bathy filled with GEBCO')
ax[2].set_title('IBCSOv2 minus GEBCO')

# Separated bathymetries:
kwags={'transform':ccrs.PlateCarree(), 'vmin':-6000, 'vmax':0, 'cmap':cmocean.cm.deep_r, 'rasterized':True}
cm1 = ax[0].pcolormesh(coord['nav_lon'][0:453,:],coord['nav_lat'][0:453,:], Bed_IBCSO['bathy'], **kwags);
cm2 = ax[1].pcolormesh(coord['nav_lon'][0:453,:],coord['nav_lat'][0:453,:], Bed_GEBCO['bathy'], **kwags);

# Difference between the bathymetries:
Bed_IBCSO_plot = xr.where(np.isnan(Bed_IBCSO['bathy']), 0, Bed_IBCSO['bathy'])
Bed_GEBCO_plot = xr.where(np.isnan(Bed_GEBCO['bathy']), 0, Bed_GEBCO['bathy'])
cm3 = ax[2].pcolormesh(coord['nav_lon'][0:453,:], coord['nav_lat'][0:453,:], Bed_IBCSO_plot - Bed_GEBCO_plot, 
                       cmap=cmocean.cm.balance, transform=ccrs.PlateCarree(), vmin=-2, vmax=2, rasterized=True);

CB = fig.colorbar(cm3, ax=ax[2], fraction=0.04)
fig.set_facecolor('white')
# fig.savefig('/home/users/birgal/data/input_data/topo/BedMachine_IBCSOv2_combined_NEMO.pdf', format='pdf')